### Programatically Start Core Machine with a Startup Script and Stop Core Machines using Paperspace API

- This notebook demonstrates how to start and stop a Paperspace Core machine using the Paperspace API.

- Make sure you have your API token created using this instruction: https://docs.digitalocean.com/reference/paperspace/api-keys/

- Also find the Paperspace API reference here: https://docs.digitalocean.com/reference/paperspace/pspace/api-reference/

### Example Use Case:
- We have setup a Paperspace Core machine running a Mistral-7B vLLM Docker container and exposes the API on Port 8000

- Say, there is no constant usage of the machine and we want to stop the machine when not in use and start it when needed.

- We can do this by using the Paperspace API as shown in the diagram below 

<!-- ![Core_PS_API Image](core_api_flow_start_stop.png) -->
<img src="core_api_flow_start_stop.png" width="600" height="450">

In [7]:
import requests
import json

api_key = "PAPERSPACE_API_KEY"
hf_token = "HF_READ_TOKEN"